In [21]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
from lxml import html
import pandas as pd
import openpyxl
import unicodedata
from mplsoccer import PyPizza, FontManager, add_image
from PIL import Image, ImageDraw, ImageOps
import matplotlib.pyplot as plt
import time

In [22]:
def getReports():

    html = urlopen('https://fbref.com/en/comps/9/stats/Premier-League-Stats')
    bs = BeautifulSoup(html, 'html.parser').encode()

    with open("data1.html", "w") as file:
        file.write(str(bs))
        

    with open("data1.html",'r') as html1:
        bs2 = BeautifulSoup(html1, 'html.parser')
        text_of_bs2 = str(bs2)
        usable_bs2 = text_of_bs2.replace("<!--"," ").replace("-->", " ")
        with open("data2.html", 'w') as commentless_content:
            commentless_content.write(usable_bs2)

    with open('data2.html', 'r') as main_page:
        bs3 = BeautifulSoup(main_page,'html.parser')
        table_contents = bs3.find_all('table')
        workbook = openpyxl.Workbook()
        sheet = workbook.active
        sheet['A1'] = "Name"
        sheet['B1'] = "Link"
        workbook.save('player_profiles.xlsx')
        for count, content in enumerate(table_contents):
            list_of_links = content.find_all('a', href= re.compile(r"^/en/players/[a-f0-9]{8}/[A-Za-z-]+$"))
            for link in list_of_links:
                if 'href' in link.attrs:
                        locA = "A"+str(count)
                        locB = "B"+str(count)
                        sheet[locA] = link.text
                        sheet[locB] = link.attrs['href']
                        workbook.save('player_profiles.xlsx')
getReports()


def name_updater():
    count = 1
    df = pd.read_excel('player_profiles.xlsx')
    while count < df.shape[0]:
        real_name = str(df['Link'].values[count]).split('/').pop().replace('-'," ")
        df['Name'].values[count] = real_name
        df.to_excel('player_profiles.xlsx')
        count+=1
name_updater()

In [24]:
def link_generator(player_name):
    count = 0
    df = pd.read_excel('player_profiles.xlsx')
    if player_name in df['Name'].values:
        b_value = df[df['Name'] == player_name]['Link'].values[0]
    return b_value


def get_players_data(player_name):
    count = 0
    player_stat_keys_raw = []
    player_stat_values = []
    global player_stat_topics
    player_stat_topics = ['Standard Stats', 'Shooting', 'Passing', 'Pass Types',
                          'Goal and Shot Creation','Defense', 'Possession','Miscellaneous Stats', 'Goalkeeping','Advanced Goalkeeping'] 
    player_data_dict = {}
    individual_components = []
    link_to_player_profile = link_generator(player_name)
    html = urlopen("https://fbref.com/"+link_to_player_profile)

    bs = BeautifulSoup(html, 'html.parser')
    global player_image_url
    player_image_url = bs.find('div', {'class':'media-item'}).find('img').attrs['src']

    scout_report_link = bs.find('div',{'class':'section_heading_text'}).find('a').attrs['href']
    
    scout_html = urlopen("https://fbref.com/"+ scout_report_link)
    
    bs_scout_all = BeautifulSoup(scout_html, 'lxml')
    bs_scout = bs_scout_all.find('div', {'id': re.compile(r'div_scout_full_')})
    stat_tables_keys = bs_scout.find("table", {'id': re.compile(r'scout_full_')} ).find_all('tr')
    stat_tables_p90Percentile = bs_scout.find("table", {'id': re.compile(r'scout_full_')}).find_all('td')

    for list_th in stat_tables_keys:
        if list_th.find('th').get_text() != "":
            if list_th.find('th').get_text() != "Statistic":
                player_stat_keys_raw.append(list_th.find('th').get_text())
  

    for list_tb_n in range(0, len(stat_tables_p90Percentile),2):
        
        if list_tb_n+1 < len(stat_tables_p90Percentile):
            list_tb = stat_tables_p90Percentile[list_tb_n]
            new_list = [list_tb.get_text()]
            new_list.append(unicodedata.normalize("NFKD",stat_tables_p90Percentile[list_tb_n+1].get_text(strip=True)))
            player_stat_values.append(new_list)
    for stat in player_stat_values:
        if "" in stat:
            player_stat_values.remove(stat)
    for stat in player_stat_keys_raw:
        if stat in player_stat_topics:
            player_stat_keys_raw.remove(stat)
    return player_stat_keys_raw, player_stat_values

In [25]:
def show_picture(params, values, name_of_player):
    font_normal = FontManager('https://raw.githubusercontent.com/google/fonts/main/apache/roboto/'
                              'Roboto%5Bwdth,wght%5D.ttf')
    font_italic = FontManager('https://raw.githubusercontent.com/google/fonts/main/apache/roboto/'
                              'Roboto-Italic%5Bwdth,wght%5D.ttf')
    font_bold = FontManager('https://raw.githubusercontent.com/google/fonts/main/apache/robotoslab/'
                            'RobotoSlab%5Bwght%5D.ttf')
    
    image = Image.open(urlopen(player_image_url))
    
    mask = Image.new('L', image.size, 0)
    draw = ImageDraw.Draw(mask)
    draw.ellipse((0, 0) + image.size, fill=255)


    masked_img = ImageOps.fit(image, mask.size, centering=(0.5, 0.5))
    masked_img.putalpha(mask)
    

    slice_colors = ["#bbEE90"] * 5 + ["#FF93ff"] * 5 + ["#FFCCCB"] * 5 + ["#87CEEB"] * 5
    text_colors = ["#000000"] * 20

    baker = PyPizza(
        params=params,                  
        background_color="#132257",     
        straight_line_color="#000000",  
        straight_line_lw=1,            
        last_circle_color="#000000",    
        last_circle_lw=1,              
        other_circle_lw=0,              
        inner_circle_size=11           
    )


    # plot pizza
    fig, ax = baker.make_pizza(
        values,                          
        figsize=(10, 12),                
        color_blank_space="same",        
        slice_colors=slice_colors,      
        value_colors=text_colors,        
        value_bck_colors=slice_colors,  
        blank_alpha=0.4,                 
        kwargs_slices=dict(
            edgecolor="#000000", zorder=2, linewidth=1
        ),                              
        kwargs_params=dict(
            color="#ffffff", fontsize=13,
            fontproperties=font_bold.prop, va="center"
        ),                               
        kwargs_values=dict(
            color="#ffffff", fontsize=11,
            fontproperties=font_normal.prop, zorder=3,
            bbox=dict(
                edgecolor="#000000", facecolor="cornflowerblue",
                boxstyle="round,pad=0.2", lw=1
            )
        )                                
    )

  
    fig.text(
        0.515, 0.945, name_of_player, size=27,
        ha="center", fontproperties=font_bold.prop, color="#ffffff"
    )


    fig.text(
        0.515, 0.925,
        "Percentile Rank vs Top-Five League Players in their Position for LAST 365 DAYS",
        size=13,
        ha="center", fontproperties=font_bold.prop, color="#ffffff"
    )

    CREDIT_2 = "inspired by: @Worville, @FootballSlices, @somazerofc & @Soumyaj15209314"
    CREDIT_3 = "Automated By @Regmi38"

    fig.text(
        0.99, 0.08, f"\n{CREDIT_2}\n{CREDIT_3}", size=15,
        fontproperties=font_italic.prop, color="#ffffff",
        ha="right"
    )

    fig.text(
        0.23, 0.9," Standard        Passing        Possession         Defense", size=18,
        fontproperties=font_bold.prop, color="#ffffff"
    )

    fig.patches.extend([
        plt.Rectangle(
            (0.205, 0.9), 0.025, 0.0196, fill=True, color="#bbEE90",
            transform=fig.transFigure, figure=fig
        ),
        plt.Rectangle(
            (0.365, 0.9), 0.025, 0.0196, fill=True, color="#FF93ff",
            transform=fig.transFigure, figure=fig
        ),
        plt.Rectangle(
            (0.505, 0.9), 0.025, 0.0196, fill=True, color="#FFCCCB",
            transform=fig.transFigure, figure=fig
        ),
        plt.Rectangle(
            (0.695, 0.9), 0.025, 0.0196, fill=True, color="#87CEEB",
            transform=fig.transFigure, figure=fig
        ),
    ])

    ax_image = add_image(
        masked_img, fig, left=0.472, bottom=0.457, width=0.086, height=0.08, zorder= -1
    )     
   
    plt.savefig(name_of_player + '@my_plot.jpg', format='jpg')
    end_time = time.time()
    time_elapsed = end_time - start_time
    print("Elapsed time: {:.2f} seconds".format(time_elapsed))

In [26]:
name_of_player = input("Enter the name")
global start_time
start_time = time.time()
def stats_gobbler():
    data_scraper = get_players_data(name_of_player)
    
    global params

    params_raw = ['Non-Penalty Goals', 'Assists', 'Goals + Assists', 'Yellow Cards', 'Red Cards', 'Passes Attempted',
                          'Pass Completion %', 'Progressive Passes', 'Through Balls','Key Passes', 'Touches','Take-Ons Attempted', 'Successful Take-Ons', 'Miscontrols',
                          'Dispossessed','Tackles','Tackles Won','Shots Blocked', 'Interceptions', 'Clearances']
    values = []

    params = ['Non-Penalty\nGoals', 'Non-Penalty\nxG', 'Goals+Assists', 'Yellow Cards', 'Red Cards', 'Passes\nAttempted',
                          'Pass\nCompletion %', 'Progressive\nPasses', 'Through Balls','Key Passes', 'Touches','Take-Ons\nAttempted', 'Successful\nTake-Ons', 'Miscontrols',
                          'Dispossessed','Tackles','Tackles Won','Shots Blocked', 'Interceptions', 'Clearances']


    df = pd.concat([pd.Series(data_scraper[0]), pd.Series(data_scraper[1])], axis=1)


    for name in params_raw:
        if name in df[0].tolist():
            value = df.loc[df[0]==name,1].iloc[0]
            values.append(int(value.pop()))
    show_picture(params, values, name_of_player)
stats_gobbler()

Enter the name Martin Odegaard


UnboundLocalError: cannot access local variable 'b_value' where it is not associated with a value